In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/agnews/train.csv')

In [4]:
df

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [7]:
# show the first 10 titles and descriptions in the dataset full text
for i in range(3):
    title = df['Title'][i]
    description = df['Description'][i]
    label = df['Class Index'][i]
    num_tokens = description.count(' ') + 1
    print('Title: ', title)
    print('Description: ', description)
    print('Number of tokens: ', num_tokens)
    print('Label: ', label)

    print('===========================================')

Title:  Wall St. Bears Claw Back Into the Black (Reuters)
Description:  Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Number of tokens:  12
Label:  3
Title:  Carlyle Looks Toward Commercial Aerospace (Reuters)
Description:  Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
Number of tokens:  30
Label:  3
Title:  Oil and Economy Cloud Stocks' Outlook (Reuters)
Description:  Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
Number of tokens:  29
Label:  3


In [8]:
tokens = []
for i in range(len(df)):    
    description = df['Description'][i]    
    num_tokens = description.count(' ') + 1
    tokens.append(num_tokens)
df['tokens'] = tokens

In [10]:
df.describe()

,Class Index,tokens
count,120000.000000,120000.000000
mean,2.500000,31.600675
std,1.118039,10.200845
min,1.000000,1.000000
25%,1.750000,25.000000
50%,2.500000,31.000000
75%,3.250000,36.000000
max,4.000000,194.000000


In [14]:

df = df[['Class Index', 'Description']]

In [15]:
df

,Class Index,Description
0,3,"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Reuters - Private investment firm Carlyle Grou...
2,3,Reuters - Soaring crude prices plus worries\ab...
3,3,Reuters - Authorities have halted oil export\f...
4,3,"AFP - Tearaway world oil prices, toppling reco..."
...,...,...
119995,1,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Red Sox general manager Theo Epstein acknowled...
119997,2,The Miami Dolphins will put their courtship of...
119998,2,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [16]:
train_df = pd.read_csv('./data/agnews/train.csv')
test_df = pd.read_csv('./data/agnews/test.csv')

train_df = train_df[['Class Index', 'Description']]
test_df = test_df[['Class Index', 'Description']]



In [18]:
import numpy as np

np.savetxt('./data/agnews/train.txt', train_df.values, fmt='%s', delimiter='\t')
np.savetxt('./data/agnews/test.txt', test_df.values, fmt='%s', delimiter='\t')

In [27]:
import pandas as pd
import numpy as np

In [55]:
kaggle_med = pd.read_csv('./data/original/kaggle_med/train.txt',delimiter='\t', header=None, names=['class', 'text'])
kaggle_med['text'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[1]) 
kaggle_med['class'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[0])

In [59]:
kaggle_med['class'].unique()

array(['22', '33', '3', '5', '38', '11', '27', '10', '20', '24', '25',
       '39', '21', '23', '29', '12', '35', '14', '16', '26', '2', '7',
       '15', '28', '8', '19', '31', '37', '32', '13', '36', '34', '4',
       '18', '1', '30', '17', '6', 'and', '9', '6.', '0', '7.'],
      dtype=object)

In [65]:
kaggle_med.describe()

,class,text
count,4255,4255
unique,43,2183
top,38,An example/template for a routine normal male...
freq,914,12


In [60]:
kaggle_med['class'].value_counts()

38     914
5      443
3      316
27     308
15     224
33     224
22     186
14     180
24     146
35     141
39     124
10      97
11      86
23      84
16      82
26      72
12      68
21      67
29      60
25      49
28      49
32      44
31      41
7       25
6       24
8       23
20      22
30      19
13      18
36      18
2       16
4       13
18      13
34       9
9        9
19       8
37       8
17       6
1        5
and      4
0        4
6.       3
7.       3
Name: class, dtype: int64

In [61]:
kaggle_med.text.loc[kaggle_med['class'] == '7.']

2808     Microscopic dissection.
3169     Microscopic dissection.
3466     Microscopic dissection.
Name: text, dtype: object

In [62]:
kaggle_med.text.loc[kaggle_med['class'] == '6.']

1073     Grade 1+ rectocele.
1468     Grade 1+ rectocele.
3595     Grade 1+ rectocele.
Name: text, dtype: object

In [63]:
kaggle_med.text.loc[kaggle_med['class'] == '0']

1504     Patient having severe sinusitis about two to ...
2341     A female for a complete physical and follow u...
3360     A 23-year-old white female presents with comp...
3558     Mother states he has been wheezing and coughing.
Name: text, dtype: object

In [64]:
kaggle_med.text.loc[kaggle_med['class'] == 'and']

732     congestive heart failure.
3574    congestive heart failure.
3792    congestive heart failure.
3939    congestive heart failure.
Name: text, dtype: object

In [68]:
new_df_train = kaggle_med.drop(kaggle_med[(kaggle_med['class'] == 'and') | (kaggle_med['class'] == '6.') | (kaggle_med['class'] == '7.')].index)


In [70]:
new_df_train.describe()

,class,text
count,4245,4245
unique,40,2180
top,38,An example/template for a routine normal male...
freq,914,12


In [71]:
np.savetxt('./data/original/kaggle_med/train.txt', new_df_train.values, fmt='%s', delimiter='\t')

In [72]:
kaggle_med = pd.read_csv('./data/original/kaggle_med/test.txt',delimiter='\t', header=None, names=['class', 'text'])
kaggle_med['text'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[1]) 
kaggle_med['class'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[0])

In [73]:
kaggle_med['class'].unique()

array(['22', '27', '5', '38', '21', '10', '6', '39', '15', '28', '14',
       '25', '12', '35', '23', '30', '26', '3', '29', '1', '33', '32',
       '16', '18', '24', '31', '7', '11', '20', '8', '9', '13', '2', '0',
       '34'], dtype=object)

In [74]:
kaggle_med.describe()

,class,text
count,472,472
unique,35,440
top,38,The patient is an 11-month-old with a diagnos...
freq,122,3


In [77]:
df = pd.read_csv('./data/original/kaggle_med/train.txt',delimiter='\t', header=None, names=['class', 'text'])

In [78]:
df

,class,text
0,22,"Occipital craniotomy, removal of large tumor ..."
1,33,CT of Lumbar Spine without Contrast. Patient...
2,33,A 68-year-old white male with recently diagno...
3,22,Falls at home. Anxiety and depression. The ...
4,3,"Nonischemic cardiomyopathy, branch vessel cor..."
...,...,...
4240,14,Esophagogastroduodenoscopy with pseudo and es...
4241,38,Belly button piercing for insertion of belly ...
4242,23,Phenol neurolysis left musculocutaneous nerve...
4243,12,Patient went out partying last night and dran...


In [84]:
duplicates = df[df.duplicated()]
len(duplicates)

31

In [85]:
print(duplicates)

      class                                               text
811      21   Noncontrast CT abdomen and pelvis per renal s...
1075     25   An example/template for a routine normal male...
1184     25   Sample normal ear, nose, mouth, and throat exam.
1535     15   An example/template for a routine normal fema...
1559     35                    Sample progress note - Gen Med.
1654      5   Occupational medicine consult with questions-...
1690     35                    Sample progress note - Gen Med.
1699     33                    MRI left knee without contrast.
1967     25   Normal review of systems template.  Negative ...
2304     15                    Sample progress note - Gen Med.
2696      3                  Sample of Pulmonary Function Test
2718     15   An example/template for a routine normal male...
2727      5   An example/template for a routine normal male...
2799     11   Sample normal ear, nose, mouth, and throat exam.
2850     18   Occupational medicine consult with questi

In [86]:
duplicates = df[df['text'].duplicated()]
len(duplicates)

2065

In [ ]:
for i in range(len(duplicates)):
    print(duplicates['text'].iloc[i])
    print('===========================================')

In [ ]:
duplicates = df.groupby('text').apply(lambda x: pd.Series({
    'class': list(x['class']),
    'count': len(x)
})).reset_index()
duplicates = duplicates[duplicates.duplicated(subset='class')]
for i in range(len(duplicates)):
    count = duplicates['count'].iloc[i]
    if count > 2:
        print('text: ',duplicates['text'].iloc[i])
        print('class: ',duplicates['class'].iloc[i])
        print('count: ',duplicates['count'].iloc[i])
        print('===========================================')

In [50]:
df = pd.read_csv('./data/original/cardio/train.txt',delimiter='\t', header=None, names=['class', 'text'])
duplicates = df[df.duplicated()]
len(duplicates)

0

## new session
### augmentation

In [1]:
import numpy as np
import pandas as pd
import textattack
from functions import *

2023-02-25 17:44:14.549474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 17:44:14.691408: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-25 17:44:15.118688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-25 17:44:15.118735: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [38]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'embedding': 'embedding_augmenter',
    'deletion': 'deletion_augmenter',
    'wordnet': 'wordnet_augmenter'
}


In [42]:
# import snoop
# @snoop(depth=2)
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')
    #data = data.sample(frac=fraction)
    #print(f'Sampled {len(data)} rows with fraction={fraction}')
    for method in method_names:
        if method not in dict_methods:
            raise ValueError(f"Method '{method}' is not supported")
        augmented_data = augment_text(
            data,
            dict_methods[method],
            fraction=fraction,
            pct_words_to_swap=pct_words_to_swap,
            transformations_per_example=transformations_per_example,
            label_column='class',
            target_column='text',
            include_original=include_original
        )
        augmented_data = augmented_data[['class', 'text']]
        np.savetxt(
            f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt',
            augmented_data.values,
            fmt='%s',
            delimiter='\t'
        )
        print(f'number of augmented samples: {len(augmented_data)}')

In [49]:
aug_samples('agnews', ['eda'], 0.001, 0.5, 2, False)

Loaded 120000 rows of data from ./data/original/agnews/train.txt


[nltk_data] Downloading package omw-1.4 to /home/peyman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


number of augmented samples: 240


In [12]:
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    methods = method_names
    for method in methods:
        augmented_data = augment_text( data, method, fraction=fraction, pct_words_to_swap=pct_words_to_swap, transformations_per_example=transformations_per_example,
                                                            label_column='class', target_column='text', include_original=include_original)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt', augmented_data.values, fmt='%s', delimiter='\t')

In [13]:
aug_samples('pc', ['checklist_augmenter'], 0.001, 0.5, 2, True)

# test

In [34]:
data = load_data(dict_datasets['pc'])
df = data.copy( )

In [35]:
df= df[~(df['text'].isin(['','none','nan','null','NaN','NULL','Null','None','NONE']))].reset_index(drop=True)

In [37]:
np.savetxt('./data/original/pc/train_clean.txt', df.values, fmt='%s', delimiter='\t')

In [41]:
aug_samples('pc_clean', ['checklist'], 0.001, 0.5, 2, True)

Loaded 39296 rows of data from ./data/original/pc/train_clean.txt
number of augmented samples: 83


In [4]:
df = pd.read_csv('./data/original/yelp/yelp.csv')
df = df[['stars', 'text']]

In [19]:
from sklearn.model_selection import train_test_split
import re
train, test = train_test_split(df, test_size=0.1, random_state=42)
train.to_csv('./data/original/yelp/train.csv', index=False)
test.to_csv('./data/original/yelp/test.csv', index=False)
# remove newlines
test['text'] = test['text'].apply(lambda x: re.sub(r'\n+', '', x))
train['text'] = train['text'].apply(lambda x: re.sub(r'\n+', '', x))

np.savetxt('./data/original/yelp/train.txt', train.values, fmt='%s', delimiter='\t')
np.savetxt('./data/original/yelp/test.txt', test.values, fmt='%s', delimiter='\t')

In [20]:
pd.read_csv('./data/original/yelp/train.txt',delimiter='\t', header=None, names=['class', 'text'])

,class,text
0,5,One bite and I was transported back to the cro...
1,3,"This place was good, just not as good as I had..."
2,5,Yum best donuts in Phoenix.
3,4,When I found out I was going to Scottsdale for...
4,4,We arrived at the store several months ago to ...
...,...,...
8995,4,Very relaxed atmosphere and employees are supe...
8996,3,07/25/11I have a new crush... and it's Christo...
8997,4,I've been frequenting the Dollar store for the...
8998,5,I love the girls there. I get my eyebrows done...
